# Grado en ciencia de datos - Big Data


# Práctica 3.2 - Clasificación

En esta práctica vamos a ver un problema de clasificación con la librería de Spark MLib.

Ten en cuenta que una vez tengas en marcha Spark, podrás visualizar la evolución de cada trabajo de Spark en  <http://localhost:4040>

In [33]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Ejemplo pySparkSQL") \
    .config("spark.sql.warehouse.dir", "file:///D:/tmp/spark-warehouse") \
    .getOrCreate()

sc = spark.sparkContext


25/12/19 15:22:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Otros imports necesarios:

In [2]:
%matplotlib inline 
from pyspark.sql import Row
from test_helper import Test
import matplotlib.pyplot as plt
from pyspark.sql.functions import *

# Construcción y ajuste de los parámetros de una Pipeline

Generalmente, una pipeline de ML incluye una serie de fases: **preprocesamiento**, **extracción de características**, **ajuste del modelo** y **validación**.

Por ejemplo, para clasificar documentos de texto tendríamos: segmentación/limpieza, extracción de características y el entrenamiento del modelo con validación cruzada para ajustar los parámetros.

Aunque existen muchas librerías para cada fase, trabajar con todas no suele ser demasiado fácil, especialmente cuando trabajamos con datasets grandes. Pero, SparkMLib facilita la labor.

En esta práctica vamos a ver como afrontar un problema de clasificación de textos simple como ejemplo de una pipeline de ML para clasificación con Spark MLlib.

In [3]:
# Imports necesarios de `spark.ml`.
from pyspark.ml import *
from pyspark.ml.param import *
from pyspark.ml.tuning import *
from pyspark.ml.feature import *
from pyspark.ml.evaluation import *
from pyspark.ml.classification import *

### El dataset "20 Newsgroups"

En este caso vamos a trabajar con una versión simplificada del dataset 20 newsgroups. Este dataset es una colección de artículos de noticias clasificadas en 20 grupos diferentes.

El dataset original se pueden encontrar en  https://archive.ics.uci.edu/ml/datasets/Twenty+Newsgroups.

Para simplificar, el que podéis encontrar en las prácticas es una versión simplificada, donde los 20 grupos los dejamos en 2. Trataremos de saber si un artículo está relacionado con ciencia o no.

El dataset que tenéis disponible está ya disponible para leerse como un DataFrame (en el original es necesario leer cada fichero y transformarlo ligeramente). El formato en el que está almacenado es Parquet, el formato binario por defecto utilizado por Spark.

Descargar el fichero y dejarlo en datos/20newsgropuBinaryFiltered/

In [4]:
import os
dbfs_dir = "./datos/20newsgropuBinaryFiltered/"
print('Ficheros en la carpeta:', os.listdir(dbfs_dir))

Ficheros en la carpeta: ['part-r-00003-ef9ac4e4-a6ba-49ff-817a-eb473f8c07de.snappy.parquet', '.part-r-00000-ef9ac4e4-a6ba-49ff-817a-eb473f8c07de.snappy.parquet.crc', '.part-r-00002-ef9ac4e4-a6ba-49ff-817a-eb473f8c07de.snappy.parquet.crc', 'part-r-00002-ef9ac4e4-a6ba-49ff-817a-eb473f8c07de.snappy.parquet', '.part-r-00001-ef9ac4e4-a6ba-49ff-817a-eb473f8c07de.snappy.parquet.crc', '_SUCCESS', 'part-r-00000-ef9ac4e4-a6ba-49ff-817a-eb473f8c07de.snappy.parquet', '.part-r-00003-ef9ac4e4-a6ba-49ff-817a-eb473f8c07de.snappy.parquet.crc', 'part-r-00001-ef9ac4e4-a6ba-49ff-817a-eb473f8c07de.snappy.parquet']


In [5]:
Test.assertEquals(len(os.listdir(dbfs_dir)), 9, 'La carpeta contiene 9 ficheros')

1 test passed.


Utiliza `spark.read.parquet` para leer la carpeta.

Posteriormente, utilizando [randomSplit](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) divide el DataFrame en el conjunto de training (60%) y el conjunto de test (40%), utilizando la semilla incluida. (Por porblemas de redondeo y truncado tenemos que usar 0.6035 y .4)

Finalmente, cachea los dos DataFrames.

In [6]:
df = spark.read.parquet(dbfs_dir)
seed = 12418
(training, test) = df.randomSplit([0.6035, 0.4], seed=seed)

training.cache()
test.cache()

DataFrame[label: int, topic: string, id: string, text: string]

In [7]:
training.count()

12124

In [8]:
# Generalmente no hay problemas con este test, pero si no coinciden los números exacto pero se parecen, seguir con la práctica.
Test.assertEquals(training.count(), 12117,  "Número de ejemplos en training incorrecto")
Test.assertEquals(test.count(), 7880, "Número de ejemplos en test incorrecto")
Test.assertEquals(training.is_cached, True, 'Training no cacheado')
Test.assertEquals(test.is_cached, True, 'Test no cacheado')

1 test failed. Número de ejemplos en training incorrecto


Exception: Número de ejemplos en training incorrecto

Vamos a ver qué forma tiene el DataFrame que tenemos disponible.

In [9]:
training.show(10)

+-----+-----------+-----+--------------------+
|label|      topic|   id|                text|
+-----+-----------+-----+--------------------+
|    0|alt.atheism|49960|From: mathew <mat...|
|    0|alt.atheism|51060|From: mathew <mat...|
|    0|alt.atheism|51120|From: mathew <mat...|
|    0|alt.atheism|51121|From: strom@Watso...|
|    0|alt.atheism|51124|From: I3150101@db...|
|    0|alt.atheism|51125|From: keith@cco.c...|
|    0|alt.atheism|51126|From: keith@cco.c...|
|    0|alt.atheism|51127|From: keith@cco.c...|
|    0|alt.atheism|51128|From: keith@cco.c...|
|    0|alt.atheism|51129|From: keith@cco.c...|
+-----+-----------+-----+--------------------+
only showing top 10 rows



Podemos explorar el dataset estudiando la distribución de tópicos existente.

Para ello crea una consulta que agrupe las noticias por `topic` y cuente cuántas noticias hay en cada uno. Almacena esta información en un DataFrame `topicCount` y muéstrala con show().

In [10]:
topicCount= training.groupBy("topic").count()
topicCount.show()

+--------------------+-----+
|               topic|count|
+--------------------+-----+
|      comp.windows.x|  615|
|        misc.forsale|  613|
|    rec.sport.hockey|  597|
|  rec.sport.baseball|  609|
|comp.os.ms-window...|  626|
|comp.sys.ibm.pc.h...|  592|
|       comp.graphics|  617|
|comp.sys.mac.hard...|  635|
|     rec.motorcycles|  599|
|           rec.autos|  603|
|         alt.atheism|  607|
|           sci.crypt|  628|
|  talk.politics.guns|  568|
|  talk.politics.misc|  587|
|soc.religion.chri...|  611|
|  talk.religion.misc|  613|
|talk.politics.mid...|  603|
|     sci.electronics|  603|
|           sci.space|  614|
|             sci.med|  584|
+--------------------+-----+



In [11]:
sorted(topicCount.collect())

[Row(topic='alt.atheism', count=607),
 Row(topic='comp.graphics', count=617),
 Row(topic='comp.os.ms-windows.misc', count=626),
 Row(topic='comp.sys.ibm.pc.hardware', count=592),
 Row(topic='comp.sys.mac.hardware', count=635),
 Row(topic='comp.windows.x', count=615),
 Row(topic='misc.forsale', count=613),
 Row(topic='rec.autos', count=603),
 Row(topic='rec.motorcycles', count=599),
 Row(topic='rec.sport.baseball', count=609),
 Row(topic='rec.sport.hockey', count=597),
 Row(topic='sci.crypt', count=628),
 Row(topic='sci.electronics', count=603),
 Row(topic='sci.med', count=584),
 Row(topic='sci.space', count=614),
 Row(topic='soc.religion.christian', count=611),
 Row(topic='talk.politics.guns', count=568),
 Row(topic='talk.politics.mideast', count=603),
 Row(topic='talk.politics.misc', count=587),
 Row(topic='talk.religion.misc', count=613)]

In [12]:
Test.assertEquals(sorted(topicCount.collect()), [(u'alt.atheism', 617), (u'comp.graphics', 582), (u'comp.os.ms-windows.misc', 568),
                                               (u'comp.sys.ibm.pc.hardware', 621), (u'comp.sys.mac.hardware', 587),
                                               (u'comp.windows.x', 610), (u'misc.forsale', 613), (u'rec.autos', 588),
                                               (u'rec.motorcycles', 617), (u'rec.sport.baseball', 612),
                                               (u'rec.sport.hockey', 618), (u'sci.crypt', 601), (u'sci.electronics', 616),
                                               (u'sci.med', 602), (u'sci.space', 599), (u'soc.religion.christian', 604),
                                               (u'talk.politics.guns', 613), (u'talk.politics.mideast', 623),
                                               (u'talk.politics.misc', 591), (u'talk.religion.misc', 635)])

1 test failed. 


Exception: 

Nuestro objetivo es predecir la etiqueta `label`, es decir, si el artículo está relacionado con la ciencia o no, es interesante observar la distribución de ejemplos por etiqueta.

Utiliza para ello el método `groupBy` seguido de `count`para obtener el número de ejemplos por etiqueta que tenemos en el dataset de train. Almacena el resultado en el DataFrame `labelCount` y utiliza show() para mostrarlo.

In [13]:
labelCount = training.groupBy("label").count()
labelCount.show()

+-----+-----+
|label|count|
+-----+-----+
|    1| 2429|
|    0| 9695|
+-----+-----+



In [14]:
Test.assertEquals(sorted(labelCount.collect()), [(0, 9699),(1, 2418)], "Conteo por tópic incorrecto")

1 test failed. Conteo por tópic incorrecto


Exception: Conteo por tópic incorrecto

### Construcción de la Pipeline para clasificar artículos de noticias

Nuestra pipline tendrá las siguientes etapas:

1. **RegexTokenizer**, tokeniza cada artículo a secuencias de palabras con un patrón de expresiones regulares,
2. **HashingTF**, mapea las secuencias de palabras producidas por RegexTokenizer a vectores de características dispersos usando hashing (no nos dentendremos en ver cómo lo hace, simplemente sabemos que coge listas de palabras y nos da un vectore de características),
3. **LogisticRegression**, entrena un modelo de regresión logísticas usando los vectores de características y las etiquetas del conjunto de entrenamiento.

<img src="http://spark.apache.org/docs/latest/img/ml-Pipeline.png" style="width: 800px;"/>

In [15]:
# Debemos construir cada fase de la Pipeline con sus parámetros y finalmente crear la Pipeline

# Tokenizer: columna de entrada = test, columna de salida = words y patrón a buscar s+ (dividimos por espacios)
tokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="s+")

# HashingTF: indicamos la columna de entrada como la de salida del tokenizer, la de salida = features y el número de caraccterísticas a obtener (5000)
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features", numFeatures=5000)

# Regresión logística con 20 iteraciones y parámetro de regularización = 0.01
lr = LogisticRegression(maxIter=20, regParam=0.01)

# Creamos la pipline de ML como una lista de fases
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [17]:
# Una vez construida la pipeline, podemos entrenar el modelo con el conjunto de training
# Utiliza fit para ajustar el modelo al training
model = pipeline.fit(training)

### Comprobar y evaluar las predicciones

Una vez obtenido el PipelineModel, queremos saber cómo se comporta.
Primero lo haremos visualizando las etiquetas predichas.

In [18]:
# Utiliza transform para predecir los resultados sobre el conjunto de training
prediction = model.transform(training)

# Muestra las etiquetas predichas junto con las reales y el testo (prediction, label y text)
# Show the predicted labels along with true labels and raw texts.
prediction.select("prediction", "label", "text").show()

+----------+-----+--------------------+
|prediction|label|                text|
+----------+-----+--------------------+
|       0.0|    0|From: mathew <mat...|
|       0.0|    0|From: mathew <mat...|
|       0.0|    0|From: mathew <mat...|
|       0.0|    0|From: strom@Watso...|
|       0.0|    0|From: I3150101@db...|
|       0.0|    0|From: keith@cco.c...|
|       0.0|    0|From: keith@cco.c...|
|       0.0|    0|From: keith@cco.c...|
|       0.0|    0|From: keith@cco.c...|
|       0.0|    0|From: keith@cco.c...|
|       0.0|    0|From: keith@cco.c...|
|       0.0|    0|From: keith@cco.c...|
|       0.0|    0|From: bobbe@vice....|
|       0.0|    0|From: bobbe@vice....|
|       0.0|    0|From: halat@pooh....|
|       0.0|    0|From: halat@pooh....|
|       0.0|    0|From: dgraham@bme...|
|       0.0|    0|From: keith@cco.c...|
|       0.0|    0|From: keith@cco.c...|
|       0.0|    0|From: rm03@ic.ac....|
+----------+-----+--------------------+
only showing top 20 rows



Parece que los resultados sobre training son buenos. Pero vamos a ver el resultado cuantitativo.

In [19]:
# Creamos un evaluador para clasificación binaria usando el área bajo la curva ROC
evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")

# Utiliza el evaluador creado con el método evaluate para obtener el AUC del modelo sobre train
evaluator.evaluate(prediction)

0.9997295869002518

El resultado en training es prácticamente perfecto (approximadamente 0.99), pero esto suele ser muchas veces una pista de que estamos sobrentrenando. Veamos el resultado sobre test.

In [20]:
# Utiliza el método evaluate del evaluador y el método transform del modelo para obtener las predicciones sobre test y posteriomente evaluarlas
evaluator.evaluate(model.transform(test))

0.9121642245871447

El AUC sobre test es mucho más pequeño (aproximadamente 0.9).
Parece que tenemos algún problema más aparte del sobreentrenamiento. 
Estudiemos las fases establecidas.

## Comprobación de la Pipeline

Las predicciones de la pipline tienen también resultados intermedios de cada fase:
* "words" del tokenizer,
* "features" del hashing ,
* "prediction", "probability", y "rawPredictions" de la regresión logística.

Veamos el esquema de "prediction".

In [21]:
prediction.printSchema()

root
 |-- label: integer (nullable = true)
 |-- topic: string (nullable = true)
 |-- id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



Podemos mirar todas las columnas usando show() sobre prediction.

¿Qué no nos cuadra?

In [22]:
prediction.show(10)

+-----+-----------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|      topic|   id|                text|               words|            features|       rawPrediction|         probability|prediction|
+-----+-----------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|alt.atheism|49960|From: mathew <mat...|[from: mathew <ma...|(5000,[10,20,40,4...|[20.2460149282553...|[0.99999999838836...|       0.0|
|    0|alt.atheism|51060|From: mathew <mat...|[from: mathew <ma...|(5000,[4,5,7,8,11...|[25.2605347161165...|[0.99999999998929...|       0.0|
|    0|alt.atheism|51120|From: mathew <mat...|[from: mathew <ma...|(5000,[8,17,78,12...|[8.96315533923012...|[0.99987197479157...|       0.0|
|    0|alt.atheism|51121|From: strom@Watso...|[from: , trom@wat...|(5000,[63,331,750...|[3.27341752604195...|[0.96350553144073...|       0.0|
|    0

Si nos fijamos bien, la columna "words" debería tener un array de strings con palabras, pero sin embargo vemos que algo no ha ido bien. Por lo que parece que el tokenizer no está funcionando

Vamos a usar `explainParams` sobre el tokenizer para ver los parámetros establecidos y su documentación.

In [23]:
print(tokenizer.explainParams())

gaps: whether regex splits on gaps (True) or matches tokens (False) (default: True)
inputCol: input column name. (current: text)
minTokenLength: minimum token length (>= 0) (default: 1)
outputCol: output column name. (default: RegexTokenizer_0d5a597774b1__output, current: words)
pattern: regex pattern (Java dialect) used for tokenizing (default: \s+, current: s+)
toLowercase: whether to convert all characters to lowercase before tokenizing (default: True)


¡Cuidado! Nos hemos olvidado de la contrabarra en la expresión regular... Debe ser "\s+" y no "s+". Vamos a corregirlo...

In [24]:
# Utiliza el método setPattern de tokenizer para poner el patrónSet the value of "pattern" back to "\s+",
# necesitarás incluir una doble contrabarra "\\s+"
tokenizer.setPattern("\\s+")

RegexTokenizer_0d5a597774b1

In [25]:
# Entrenamos el modelo de nuevo
model = pipeline.fit(training)

In [26]:
# Comprobamso las predicciones y que las palabras tiene buena pinta
prediction = model.transform(training)
prediction.show(10)

+-----+-----------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|      topic|   id|                text|               words|            features|       rawPrediction|         probability|prediction|
+-----+-----------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|alt.atheism|49960|From: mathew <mat...|[from:, mathew, <...|(5000,[4,7,13,20,...|[10.4042478399151...|[0.99996969743340...|       0.0|
|    0|alt.atheism|51060|From: mathew <mat...|[from:, mathew, <...|(5000,[1,4,7,11,1...|[26.1949956196838...|[0.99999999999579...|       0.0|
|    0|alt.atheism|51120|From: mathew <mat...|[from:, mathew, <...|(5000,[20,54,55,9...|[1.96478904847501...|[0.87705030342612...|       0.0|
|    0|alt.atheism|51121|From: strom@Watso...|[from:, strom@wat...|(5000,[41,42,54,8...|[2.75807605910461...|[0.94036783815472...|       0.0|
|    0

In [27]:
# Evaluamos el modelo en train y en test
evaluator.evaluate(prediction)

0.9997749176138168

In [28]:
evaluator.evaluate(model.transform(test))

0.9648382870178933

Ahora sí que parece que funciona mejor, aunque seguimos teniendo algo de sobrentrenamiento (0.999 train y 0.96 test).
Para mejorar el error de generalización podemos ajustar los parámetros de la Pipeline.

## Ajuste de parámetros mediante validación cruzada

Podemos usar la validación cruzada en MLlib mediante `CrossValidator`.

Se toma una lista de combinaciones de parámetros y una medida de evaluación y él automáticamente busca la mejor combinación mediante validación cruzada.

Consulta la siguiente documentación para preparar la validación cruzada.

[CrossValidator](http://spark.apache.org/docs/2.0.2/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator)

[ParamGridBuilder](http://spark.apache.org/docs/2.0.2/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder)

In [29]:
# Las combinaciones de parámetros se generan como todas las posibles combinaciones entre los diferentes parámetros establecidos
# Para simplificarlo, solo usaremos diferentes parámetros de hashing TF y del parámetro de regularización de la regresión
# Utiliza ParamGridBuilder para considerar los siguientes parámetros
# hashingTF.numFeatures = [1000, 10000]
# lr.regParam = [0.05, 0.2]
paramGrid = ParamGridBuilder() \
  .addGrid(hashingTF.numFeatures, [1000, 10000]) \
  .addGrid(lr.regParam, [0.05, 0.2]) \
  .build()

# Creamos un CrossValidator para ajustar la pipeline
# Utilizamos como estimator la pipeline, como evaluator el evaluator ya definido y como parámetros paramGrid.
# Para que tarde menos utiliza solo 2 particiones
cv = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=2)

Ajustar un modelo de validación cruzada funciona igual que hacerlo con la Pipeline. Llevará más tiempo porque se ajustan muchos más modelos para elegir el mejor.

In [30]:
# Utiliza fit sobre cv para obtener el modelo entrenado con el conjunto de training
cvModel = cv.fit(training)

Veamos los resultados que obtenemos ahora con el evaluador.

In [31]:
# Primero sobre el conjunto training
evaluator.evaluate(cvModel.transform(training))

0.9990963582345102

In [32]:
# Ahora sobre el conjunto test
evaluator.evaluate(cvModel.transform(test))

0.979532024933837

¡Hemos mejorado en test! (0.9756 aprox.)
Solo hemos probado unas pocas combinaciones de parámetros. Podríamos realizar muchos más experimentos para mejorar todavía más los resultados.